In [ ]:
import pylab
import pymoc
import xidplus

import numpy as np
%matplotlib inline
from astropy.table import Table
import os
import seaborn as sns

This notebook uses all the raw data from the Blind SPIRE catalogue to create XID+ prior object and relevant tiling scheme

## Read in XID+SPIRE catalogue

In [ ]:
loc = 'dmu_products/dmu22/'
name_field= ['AKARI-SEP_SPIRE','EGS_SPIRE','ELAIS-S1_SPIRE','SA13_SPIRE','XMM-13hr_SPIRE',\
            'COSMOS_SPIRE','ELAIS-N2_SPIRE','HDF-N_SPIRE','SPIRE-NEP_SPIRE','xFLS_SPIRE','ELAIS-N1_SPIRE',\
            'XMM-LSS_SPIRE','Lockman-SWIRE_SPIRE','CDFS-SWIRE_SPIRE', 'GAMA-09_SPIRE','GAMA-12_SPIRE',\
            'GAMA-15_SPIRE','SSDF_SPIRE','Bootes_SPIRE','HATLAS-NGP_SPIRE','HATLAS-SGP_SPIRE']
which_field = 19
name_field = name_field[which_field] # select a field, in this case SA13
print(name_field)
XID_table=Table.read(loc+name_field+'_all.fits')

In [ ]:
XID_table[0:10]

The uncertianties become Gaussian by $\sim 20 \mathrm{\mu Jy}$

## Read in Maps

In [ ]:
loc = 'dmu19/dmu19_HELP-SPIRE-maps'
name = [name_field+'250_v1.0.fits',name_field+'350_v1.0.fits',name_field+'500_v1.0.fits']
pswfits=loc+name[0]
pmwfits=loc+name[1]
plwfits=loc+name[2]
os.mkdir('./OUT_'+name_field[0:-6])
output_folder ='./OUT_'+name_field[0:-6]+'/'


In [ ]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim250=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist[1].header)
pixsize250 = hdulist["Matchedfilter"].header["PIXSIZE"] 

hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim350=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist[1].header)
pixsize350 = hdulist["Matchedfilter"].header["PIXSIZE"] 
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header
im500=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim500=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist[1].header)
pixsize500 = hdulist["Matchedfilter"].header["PIXSIZE"] 

hdulist.close()

In [ ]:
## Set XID+ prior class
ID = np.arange(1,np.size(XID_table['RA'])+1)
ID = ID.astype(str)

In [ ]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(XID_table['RA'],XID_table['DEC'],name_field+'_250_XID.fits',ID=ID)#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu)
prior350.prior_cat(XID_table['RA'],XID_table['DEC'],name_field+'_350_XID.fits',ID=ID)#Set input catalogue
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu)
prior500.prior_cat(XID_table['RA'],XID_table['DEC'],name_field+'_500_XID.fits',ID=ID)#Set input catalogue
prior500.prior_bkg(-5.0,5)


In [ ]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [ ]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines

order=7 
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
#output_folder='./'
outfile=output_folder+'Master_prior.pkl'

with open(outfile, 'wb') as f:
    xidplus.io.pickle.dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},f)
    
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)


In [ ]:
print(output_folder)

## The Master_prior.pkl and Tiles.pkl are used to run XID+
# Running on Apollo


% mv XID\_plus\_hier.sh OUT\_"name_field"

% cd OUT_"name_field"

% module load sge

% qsub -t 1-$n_hier -q mps.q -jc mps.short XID_plus_hier.sh 

### n_hier is the number of large tiles

% cd ..

% qsub -t 1-$n_tiles -pe openmp 4 -l h_rt=6:00:00 -l m_mem_free=13G -q mps.q XID_plus_tile.sh
### n_hier is the number of small tiles
### Then combine the Bayesian maps into one:

% python make_combined_map.py

% cd output

% ls *cat.fits > cat_files 

% module load starlink/hikianalia-64bit

% stilts tcat ifmt=fits in=@cat_files out=dmu22_XID+SPIRE_"name_field"_BLIND.fits



## The data products are:
### dmu22XID+SPIRE"name_field"_BLIND.fits
#### dmu22_XID+SPIRE\_psw_"name_field"_Bayes_Pval
#### dmu22_XID+SPIRE\_pmw_"name_field"_Bayes_Pval
#### dmu22_XID+SPIRE\_plw_"name_field"_Bayes_Pval

# Final validation of the data can be found at: 
# http://hedam.lam.fr/HELP/data/dmu_products/dmu22/
# dmu22_"name\_field"/XID+BLIND_"name_field"_final_processing.ipynb